In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
from sklearn import metrics
import time

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from keras.layers import Input, Dense, Convolution2D, \
    MaxPooling2D, UpSampling2D, BatchNormalization, Dropout, \
    UpSampling2D, Layer, Flatten, Activation, Reshape, GRU
from keras.models import Model, Sequential
from keras import backend as K
from keras.layers.advanced_activations import ELU
from keras.callbacks import TensorBoard

In [3]:
RAND_SEED = 777
np.random.seed(RAND_SEED)

In [4]:
IMG_SHAPE = (96, 2048, 1)
N_CH = 16

# axis 0 is batch sample index
AXIS_FREQ = 1
AXIS_TIME = 2
AXIS_CH = 3

VAL_DATA_FILE = 'out/val-msgs.mem'
VAL_YS_FILE = 'out/val-ys.mem'

N_TRAIN_FILES = 1
TRAIN_DATA_FILE = 'out/train-msgs-%d.mem'
TRAIN_YS_FILE = 'out/train-ys-%d.npy'

MODEL_PATH = 'out/models/c-f%d-e%d-auc%.5f'

In [5]:
# load validation data
# X_val = np.memmap(VAL_DATA_FILE, dtype=np.float32, mode='r+').reshape((-1,) + IMG_SHAPE)
# y_val = np.fromfile(VAL_YS_FILE, dtype=np.float32).reshape(-1, 2)

In [6]:
scores = []

In [7]:
def score_auc():
    s = 0
    n = X_val.shape[0]
    
    print '\n'
    y_p = model.predict(X_val[s:s+n], verbose=True, batch_size=32)
    y_p = np.nan_to_num(y_p)
    return metrics.roc_auc_score(y_val[s:s+n].T[0], y_p.T[0])

In [8]:
class MyCallback(keras.callbacks.Callback):
    def _validate(self, epoch):
        s = score_auc()
        scores.append(s)
        np.save('out/scores.npy', scores)
        print "\n\n AUC = %.5f\n"%(s)
        if True or len(scores) == 0 or s >= max(scores):
            f = MODEL_PATH%(train_file_n, epoch, s)
            print 'Saving to: ', f, '\n'
            model.save(f)
#     def on_train_begin(self, epoch, logs={}):
#         self._validate(0)
    def on_epoch_end(self, epoch, logs={}):
        self._validate(1 + epoch)
    def on_epoch_begin(self, epoch, logs={}):
        print '\n'

In [9]:
model = Sequential()

model.add(BatchNormalization(axis=AXIS_TIME, input_shape=IMG_SHAPE))

# conv 1
model.add(Convolution2D(64, 5, 5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(1,4)))

# conv 2
model.add(Convolution2D(128, 5, 5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,4)))

# conv 3
model.add(Convolution2D(128, 5, 5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,4)))

# conv 4
model.add(Convolution2D(192, 5, 5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,4)))

# conv 5
model.add(Convolution2D(256, 5, 5, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Reshape((6*4,256)))

model.add(GRU(32, return_sequences=True))
model.add(GRU(32, return_sequences=False))

model.add(Dropout(0.5))

model.add(Dense(2, activation='sigmoid'))
model.add(Activation('softmax'))

In [10]:
# model.output_shape

In [ ]:
model.compile(optimizer='SGD', loss='binary_crossentropy')

In [ ]:
for train_file_n in range(N_TRAIN_FILES):

    X_file = TRAIN_DATA_FILE%(1 + train_file_n)
    y_file = TRAIN_YS_FILE%(1 + train_file_n)
    
    print 'Using file', X_file
    
    X_train = np.memmap(X_file, dtype=np.float32, mode='r+')
    X_train = X_train.reshape((-1,) + IMG_SHAPE)
    y_train = np.fromfile(y_file, dtype=np.float32)
    y_train = y_train.reshape(-1, 2)
    
    y_train = np.repeat(y_train, N_CH, axis=0)
    
#     X_train = X_train[:100]
#     y_train = y_train[:100]
    
    model.fit(
        X_train,
        y_train,
        batch_size=32,
        nb_epoch=75,
#         validation_data=(X_val, y_val),
        shuffle=True,
        callbacks=[
            keras.callbacks.ModelCheckpoint(
                'out/models/c-e{epoch:02d}-l{loss:.5f}.hdf5', monitor='loss', verbose=1,
                save_best_only=False, save_weights_only=False, mode='auto'),
            TensorBoard(log_dir='/tmp/tf-mls-c')
        ]
    )

    del X_train
    del y_train

Using file out/train-msgs-1.mem
Epoch 1/75
96672/96672 [==============================] - 4163s - loss: 0.4040  
Epoch 2/75
82784/96672 [========================>.....] - ETA: 602s - loss: 0.3898

In [ ]:
y_train.shape

In [ ]:
import matplotlib.cm as cm
plt.imshow(np.flipud(d[0]), aspect=10)

In [4]:
np.repeat([[1,2],[3,4]], 3, axis=0)

array([[1, 2],
       [1, 2],
       [1, 2],
       [3, 4],
       [3, 4],
       [3, 4]])

In [18]:
X_train.shape

(96672, 96, 2048, 1)

In [19]:
y_train.shape

(96672, 2)